In [1]:
import json
import copy as copy
import numpy as np
from tqdm import tqdm
from transformers import BertModel

bert_model = "E:\\hugging_face\\bert-base-chinese"
import char_until
from transformers import AutoTokenizer


In [2]:
tokenizer = AutoTokenizer.from_pretrained(bert_model)

In [3]:
def search(arr, span):
    start_index = -1
    span_len = len(span)
    for i in range(len(arr)-span_len):
        if arr[i:i+span_len] == span:
            start_index = i
            break
    end_index = start_index + len(span)
    return start_index, end_index

In [4]:
p_entitys = ['丈夫', '上映时间', '主持人', '主演', '主角', '作曲', '作者', '作词', '出品公司', '出生地', '出生日期',
             '创始人', '制片人', '号', '嘉宾', '国籍', '妻子', '字', '导演', '所属专辑', '改编自', '朝代', '歌手',
             '母亲', '毕业院校', '民族', '父亲', '祖籍', '编剧', '董事长', '身高', '连载网站']

In [ ]:
max_length = 300
token_list = []
label_list = []
with open("../data/train_data.json", "r", encoding="utf-8") as f:
    data = json.load(f)
    for line in tqdm(data):
        text = line['text']
        new_spo_list = line['new_spo_list']
        for spo in new_spo_list:
            s_entity = spo["s"]["entity"]
            p_entity = spo["p"]["entity"]
            o_entity = spo["o"]["entity"]
            
            text_p = p_entity + "[SEP]" + text
            token = tokenizer.encode(text_p)
            label = [0] * len(token)
            p_entity_token = p_entitys.index(p_entity)
            
            s_label = (2 * p_entity_token) - 1
            o_label = (2 * p_entity_token)
            
            s_entity_token = tokenizer.encode(s_entity)[1:-1]
            o_entity_token = tokenizer.encode(o_entity)[1:-1]
            s_entity_start_index, s_entity_end_index = search(token, s_entity_token)
            if s_entity_start_index != -1:
                for j in range(s_entity_start_index, s_entity_end_index):
                    label[j] = s_label
                    
            o_entity_start_index, o_entity_end_index = search(token, o_entity_token)
            if o_entity_start_index != -1:
                for j in range(o_entity_start_index, o_entity_end_index):
                    label[j] = o_label
                    
            token_list.append(token)
            label_list.append(label)
            
                

 16%|█▌        | 18339/113613 [00:10<00:55, 1731.45it/s]

In [ ]:
print(len(token_list), len(label_list))

In [ ]:
import tensorflow as tf
from transformers import AutoTokenizer, TFBertModel


In [ ]:
strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"],
                                          cross_device_ops=tf.distribute.ReductionToOneDevice())
with strategy.scope():
    tokenizer = AutoTokenizer.from_pretrained(bert_model)
    model = TFBertModel.from_pretrained(bert_model)
    input_token = tf.keras.Input(shape=(max_length,), dtype=tf.int32)
    embedding = (model(input_token)[0])
    embedding = tf.keras.layers.Dropout(0.217)(embedding)
    output = tf.keras.layers.Dense(70, activation="softmax")(embedding)
    model = tf.keras.Model(inputs=input_token, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])
    import get_data
    batch_size = 4
    for i in range(20):
        model.fit(get_data.generator(batch_size), steps_per_epoch=get_data.train_length//batch_size, epochs=1)
        model.save_weights("../saver/bert_ner.h5")